In [1]:
%%time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [89]:
# machine specifications:
import platform
print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(platform.processor())

x86_64
#1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
Linux-6.1.58+-x86_64-with-glibc2.35
uname_result(system='Linux', node='2b13a760ca66', release='6.1.58+', version='#1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023', machine='x86_64')
Linux
x86_64


In [2]:
%%time
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install PyMuPDF
!pip install python-docx
!pip install beautifulsoup4 lxml
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!pip install tensorflow
!pip install sentence_transformers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not u

In [48]:
%%time
import json
import re
import fitz
from docx import Document
from bs4 import BeautifulSoup
import pytesseract
from PIL import Image
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [4]:
%%time
def doc2text(file_path):
    if file_path.endswith('pdf'):
        with fitz.open(file_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
        return text

    elif file_path.endswith('docx'):
        document = Document(file_path)
        text = []
        for paragraph in document.paragraphs:
            text.append(paragraph.text)
        return '\n'.join(text)

    elif file_path.endswith('html'):
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        soup = BeautifulSoup(html_content, 'lxml')
        return soup.get_text(separator='\n', strip=True)

    elif file_path.endswith('jpg') or file_path.endswith('jpeg') or file_path.endswith('png'):
      image = Image.open(file_path)
      extracted_text = pytesseract.image_to_string(image)
      return extracted_text

# Resume Parsing

In [81]:
%%time
sec = [[#skills
        'areas of experience',
        'areas of expertise',
        'areas of knowledge',
        'skills',

        "other skills",
        "other abilities",
        'career related skills',
        'professional skills',
        'specialized skills',
        'technical skills',
        'soft skills',
        'computer skills',
        'personal skills',
        # 'computer knowledge',
        # 'technologies',

        'proficiencies',
        'languages',
        'language competencies and skills',
        'programming languages',
        'competencies'
],
      [#experience
        'employment history',
        'employment data',
        'career summary',
        'work history',
       'technical experience',
        'working history',
        'work experience',
        'experience',
        'professional experience',
        'professional background',
        'professional employment',
        'additional experience',
        'career related experience',
        "professional employment history",
        'related experience',
        'relevant experience',
        'programming experience',
        'freelance',
        'freelance experience',
        'internship experience',
        'internships',
        'apprenticeships',
        'army experience',
        'military experience',
        'military background',
               'special training',
       'training'

],
[#education
        'academic background',
        'academic experience',

        'courses',
        'qualification',
        'related courses',
        'education',
        'educational background',
        'educational qualifications',
        'educational training',
        'education and training',
        'academic training',
        'Academic Qualification',
        #'professional training',
        'course project experience',
        'related course projects',
        'college activities',
        'certifications',
    ],



[#miscellaneous
        'activities and honors',
        #'activities',
        'affiliations',
        'professional affiliations',
        'associations',
        'professional associations',
        'memberships',
        'professional memberships',
        'athletic involvement',
        'community involvement',

        'civic activities',
        'extra-Curricular activities',
        'professional activities',
        'volunteer work',
        'volunteer experience',
        'additional information',

],

[#achievements
        'achievement',
        'achievements',
        'awards and achievements',
        'licenses',
        'license',

        'conference presentations',
        'conventions',
        'dissertations',
        'exhibits',
        'papers',
        'publications',
        'professional publications',
        'research experience',
        'research grants',


        'research projects',
        'personal projects',
        'current research interests',

]
]

section_names=["skills","experience","education","miscellaneous","achievements"]
check_segmented = [
    False,
    False,
    False,
    False,
    False,

]

def classify_sections(text):
    # custom logic for classifying sections (similar to the previous example)
    sections = {}
    current_section = None
    temp=False
    for line in text.split('\n'):
     # print(line)
      temp=False
      for section_keywords in sec:
        if  (check_segmented[sec.index(section_keywords)]==False) and any((keyword.lower() in line.lower())  for keyword in section_keywords):
            for i in range(len(check_segmented)):
              check_segmented[i]=False
            check_segmented[sec.index(section_keywords)] = True

            temp=True
            current_section =section_names[sec.index(section_keywords)]
            if sections.get(current_section,0)!=0:
              continue
            sections[current_section] = []
            break
      if current_section !=None and temp==False:
        sections[current_section].append(line)


    return sections

def parse_resume(path):
    text = doc2text(path)
    sections = classify_sections(text)
    return text, sections


In [95]:
%%time
import pandas as pd
df=pd.DataFrame(columns=section_names)
df

CPU times: user 3.24 ms, sys: 982 µs, total: 4.22 ms
Wall time: 4.25 ms


,skills,experience,education,miscellaneous,achievements


In [97]:
%%time
import os
resume_content=''
for filename in os.listdir("/content/drive/MyDrive/hackathon/resume-data"):

    resume_content, sections = parse_resume('/content/drive/MyDrive/hackathon/resume-data/Charles Obuseh.pdf')
    df=df.append(sections,ignore_index=True)
    print("Resume Sections:")
    for section, content in sections.items():
      print(section.capitalize() + "   :")

      print('\n'.join(content))

      print("-----------------")
    break
    print("******************************************************************")

Resume Sections:
Skills   :
software, languages, and tools with the ability to learn new tools, databases and systems to maintain/enhance strategic vision of 
the organization and able to provide cutting-edge data engineering skills. Seeking a reputable organization to contribute to with 
opportunities for personal growth.  
. TECHNICAL SKILLS 
Languages 
• 
Python, SQL 
Others 
• 
Git, Bash 
 
Data Manipulation & 
Visualization  
• 
Pandas, Tableau BI, ETL, 
Apache Spark, Hadoop 
HDFS, Snowflake Data 
Factory, SnowSQL and 
Snowpipe, Amazon Kinesis 
Firehose, Databricks, AWS 
EMR, Athena, Lambda, 
Amazon MW Apache 
Airflow and AWS EC2  
QuickSight and SNS, 
Cloudera infrastructure – 
Ambari, Hortonworks 
Sandbox and Zeppelin 
Notebook 
 
Databases and Storage 
• 
AWS RDS, MySQL, 
MongoDB, AWS 
DynamoDB, 
SQLAlchemy, 
PostgreSQL, AWS S3 
Buckets, SQLite, Azure, 
Oracle. 
 
 
  
 
 
 
Excellent project and time manager with outstanding excellent presentation skills, a great team player a

<timed exec>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [98]:
%%time
# Define a function to concatenate skill and experience for each row
def concatenate_skills_experience(row):
    # Convert the list of skills to a string
    skills_str = ', '.join(row['skills']) if isinstance(row['skills'], list) else str(row['skills'])
    # Convert the element in the 'experience' column to a string
    experience_str = ', '.join(row['experience']) if isinstance(row['experience'], list) else str(row['experience'])
    return skills_str + ', ' + experience_str

# Create a new DataFrame 'x' with the 'skills' and 'experience' columns
x = df[['skills', 'experience']].copy()

# Apply the concatenate_skills_experience function to each row and assign the result to column 'y'
x['y'] = x.apply(concatenate_skills_experience, axis=1)

# Drop the original 'skills' and 'experience' columns
x.drop(['skills', 'experience'], axis=1, inplace=True)

x


CPU times: user 5.86 ms, sys: 0 ns, total: 5.86 ms
Wall time: 5.91 ms


,y
0,"software, languages, and tools with the abilit..."


# Extract Noun Phrases

In [99]:
%%time
import spacy
import pandas as pd

# Load the English language model in SpaCy
nlp = spacy.load("en_core_web_sm")

# Function to extract noun phrases from text using SpaCy
def extract_noun_phrases(text):
    doc = nlp(text)
    noun_phrases = [chunk.text for chunk in doc.noun_chunks]
    return noun_phrases

# Load the DataFrame 'x' created in the previous code
# Replace 'x.csv' with the actual file path if saving/loading from a file


# Extract noun phrases from the 'y' column of DataFrame 'x'
noun_phrases = []
for text in x['y']:
    noun_phrases.extend(extract_noun_phrases(text))

noun_phrases

CPU times: user 767 ms, sys: 12 ms, total: 779 ms
Wall time: 920 ms


['software',
 'languages',
 'tools',
 'the ability',
 'new tools',
 'databases',
 'systems',
 'enhance strategic vision',
 'the organization',
 'cutting-edge data engineering skills',
 'a reputable organization',
 ', opportunities',
 'personal growth',
 'TECHNICAL SKILLS',
 'Languages',
 'Python',
 'SQL',
 'Others',
 'Git',
 'Bash',
 'Data Manipulation',
 'Visualization',
 'Pandas',
 'Tableau BI',
 'ETL',
 'Apache Spark',
 'Hadoop',
 'HDFS',
 'Snowflake Data',
 'Factory',
 'SnowSQL',
 'Snowpipe',
 'Amazon Kinesis',
 'Firehose',
 'Databricks',
 'AWS',
 'EMR',
 'Athena',
 'Lambda',
 'Amazon MW Apache',
 'Airflow',
 'AWS',
 'EC2',
 'QuickSight',
 'SNS',
 'Cloudera infrastructure',
 'Ambari',
 'Hortonworks',
 'Sandbox',
 'Zeppelin',
 'Notebook',
 'Databases',
 'Storage',
 'AWS RDS',
 'MySQL',
 'AWS',
 'DynamoDB',
 'SQLAlchemy',
 'PostgreSQL',
 'Buckets',
 'SQLite',
 'Azure',
 'Oracle',
 ',  ,  ,   ,  ,  ,  , Excellent project and time manager',
 'outstanding excellent presentation skills',

In [100]:
%%time
import pandas as pd
df1 = pd.read_csv(r'/content/drive/MyDrive/hackathon/noun_phrases_1.csv')
df2 = pd.read_csv(r'/content/drive/MyDrive/hackathon/noun_phrases_2.csv')


# Append df2 to df1
merged_df = df1.append(df2)
merged_df

CPU times: user 11 ms, sys: 1 ms, total: 12 ms
Wall time: 17.9 ms


<timed exec>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,noun_phrase,skills/non_skills
0,management decision-making,skills
1,Power bi,skills
2,Tableau,skills
3,visualization tools,skills
4,Snowflake,skills
...,...,...
1494,database,non_skills
1495,direct marketing,non_skills
1496,the firm’s top clients,non_skills
1497,nature,non_skills


In [101]:
%%time
merged_df = merged_df.sample(frac=1, random_state=42)
merged_df.reset_index(drop=True, inplace=True)

CPU times: user 2.32 ms, sys: 21 µs, total: 2.34 ms
Wall time: 2.34 ms


In [102]:
%%time
merged_df['noun_phrase']

CPU times: user 216 µs, sys: 12 µs, total: 228 µs
Wall time: 236 µs


0                           portfolio
1                                 Key
2                   various locations
3                                   ●
4                    Google Workspace
                    ...              
2493                              Apr
2494                          MongoDB
2495                      A/B Testing
2496    Hybrid Neural Network Library
2497        the subject matter expert
Name: noun_phrase, Length: 2498, dtype: object

# Skill Extraction

In [103]:
%%time
mapping = {'skills': 1, 'non_skills': 0}

merged_df[' skills/non_skills'] = merged_df[' skills/non_skills'].map(mapping)


CPU times: user 3.59 ms, sys: 0 ns, total: 3.59 ms
Wall time: 5.63 ms


In [104]:
%%time
merged_df

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 8.58 µs


,noun_phrase,skills/non_skills
0,portfolio,0
1,Key,0
2,various locations,0
3,●,0
4,Google Workspace,1
...,...,...
2493,Apr,0
2494,MongoDB,1
2495,A/B Testing,1
2496,Hybrid Neural Network Library,1


In [105]:
%%time
merged_df['noun_phrase'] = merged_df['noun_phrase'].astype(str)

CPU times: user 970 µs, sys: 0 ns, total: 970 µs
Wall time: 985 µs


# Model Training for skill extraction

In [106]:
%%time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import SpatialDropout1D, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
# Tokenize the noun phrases
tokenizer = Tokenizer()
tokenizer.fit_on_texts(merged_df['noun_phrase'])
X_sequences = tokenizer.texts_to_sequences(merged_df['noun_phrase'])
print(X_sequences)
# Pad sequences to ensure uniform length
max_length = 100
X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_padded, merged_df[' skills/non_skills'], test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    # Flatten(),
    Dense(units=10, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


[[206], [310], [53, 431], [18], [83, 748], [207], [162, 311, 312, 208], [54], [749, 73, 1], [13, 128], [84, 48], [209], [129, 210], [750, 751], [163], [11, 432, 211], [752, 753], [433, 754, 23, 28, 3, 755, 433, 756, 212, 63], [74, 757], [5, 758, 33], [434, 435], [213, 19], [164], [759, 214], [760, 75], [19], [761, 436, 8], [9], [24, 64], [2, 39], [762, 763, 437, 764], [765, 438, 40], [4, 165], [439], [83, 440], [34, 25], [766, 313], [14, 10], [767, 314, 768], [5, 769, 311, 312, 208], [164], [1, 26, 165], [441], [49, 442, 106, 34, 25], [166, 167], [315], [1, 107, 85], [316, 130, 443, 130], [9], [770], [771], [41], [772, 773, 774], [215], [775, 1, 216], [444, 445, 776], [1], [777], [6], [86], [317], [14, 10], [2, 40], [42], [87], [88, 217], [43], [13, 76], [22], [131], [778], [55, 446, 779, 780, 218], [318, 168, 7, 219, 319, 781, 447, 782, 783, 447, 448], [220], [784], [27], [132, 785], [221], [108, 222], [786, 320], [4], [2, 8], [35], [787], [2, 133, 65], [3, 134], [2, 1], [56], [788], 

In [107]:
%%time
model.save('/content/drive/MyDrive/hackathon/model.h5')

CPU times: user 31.7 ms, sys: 1.55 ms, total: 33.2 ms
Wall time: 54.7 ms


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Skill Extraction

In [108]:
%%time
text='\n'.join(df['skills'][0])
# text+=df['experience'][0]
text+='\n'.join(df['experience'][0])
text

CPU times: user 0 ns, sys: 394 µs, total: 394 µs
Wall time: 400 µs


'software, languages, and tools with the ability to learn new tools, databases and systems to maintain/enhance strategic vision of \nthe organization and able to provide cutting-edge data engineering skills. Seeking a reputable organization to contribute to with \nopportunities for personal growth.  \n. TECHNICAL SKILLS \nLanguages \n• \nPython, SQL \nOthers \n• \nGit, Bash \n \nData Manipulation & \nVisualization  \n• \nPandas, Tableau BI, ETL, \nApache Spark, Hadoop \nHDFS, Snowflake Data \nFactory, SnowSQL and \nSnowpipe, Amazon Kinesis \nFirehose, Databricks, AWS \nEMR, Athena, Lambda, \nAmazon MW Apache \nAirflow and AWS EC2  \nQuickSight and SNS, \nCloudera infrastructure – \nAmbari, Hortonworks \nSandbox and Zeppelin \nNotebook \n \nDatabases and Storage \n• \nAWS RDS, MySQL, \nMongoDB, AWS \nDynamoDB, \nSQLAlchemy, \nPostgreSQL, AWS S3 \nBuckets, SQLite, Azure, \nOracle. \n \n \n  \n \n \n \nExcellent project and time manager with outstanding excellent presentation skills, a gr

In [109]:
%%time
def extract_skills(text):
    noun_phrases = extract_noun_phrases(text)
    noun_phrases = np.array(noun_phrases).astype(str)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(noun_phrases)
    X_sequences = tokenizer.texts_to_sequences(noun_phrases)

    # Pad sequences to ensure uniform length
    max_length = 100
    X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')

    # Reshape input data to add embedding dimension
    X_padded = np.expand_dims(X_padded, axis=-1)  # Add embedding dimension

    predictions = model.predict(X_padded)

    # Filter predicted skills based on the threshold
    threshold = 0.5  # Adjust as needed
    predicted_skills = [noun_phrases[i] for i, pred in enumerate(predictions) if pred > threshold]

    return predicted_skills


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


In [110]:
%%time
extracted_skills = extract_skills(text)

5/5 [==============================] - 0s 14ms/step
CPU times: user 331 ms, sys: 7.69 ms, total: 338 ms
Wall time: 382 ms


In [111]:
%%time
print(f'Extracted Skills: {extracted_skills}')

Extracted Skills: ['software', 'languages', 'tools', 'new tools', 'databases', 'enhance strategic vision', 'opportunities', 'personal growth', 'TECHNICAL SKILLS \nLanguages \n• \nPython', 'SQL \nOthers \n• \nGit', 'Pandas', 'Hadoop \nHDFS', 'Snowflake Data \nFactory', 'Amazon Kinesis \nFirehose', 'Databricks', 'AWS \nEMR', 'Athena', 'Lambda', 'Amazon MW Apache \nAirflow', 'Hortonworks \nSandbox', 'Zeppelin \nNotebook \n \nDatabases', 'Storage', 'MySQL', 'AWS \nDynamoDB', 'SQLAlchemy', 'AWS S3 \nBuckets', 'Azure', 'Oracle', 'Excellent project and time manager', 'outstanding excellent presentation skills', 'a great team player', 'Data Engineer | TC Energy', 'Present | Odfjell Terminal', 'TX | 02/2019', '09/2019 | Enterprise \nProducts', 'CF', 'process data pipeline', 'asset data migration', 'Amazon Kinesis', 'Firehose', 'storage', 'Snowflake', 'bucket', 'Amazon Managed Workflows', 'Programming language', 'Python', 'SparkSql', 'data pipeline', 'Zeppelin Notebook', 'Programming language', 

In [112]:
%%time
final_skills = '\n'.join(extracted_skills)

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 19.3 µs


# Abstractive Skill Extraction

In [117]:
%%time
nlp_model = spacy.load('/content/drive/MyDrive/hackathon/skills_model')
# text=df['skills']
doc = nlp_model(final_skills)
abstractive_skills=''
for ent in doc.ents:
  abstractive_skills+=ent.text
  print(f'{ent.label_.upper()}->\n {ent.text}')

SKILLS->
 Languages 
• 
Python
SQL 
Others 
• 
Git
Pandas
Hadoop 
HDFS
Snowflake Data 
Factory
Amazon Kinesis 
Firehose
Databricks
AWS 
EMR
Athena
Lambda
Amazon MW Apache 
Airflow
Hortonworks 
Sandbox
Zeppelin 
Notebook 
 
Databases
Storage
MySQL
AWS 
DynamoDB
SQLAlchemy
AWS S3 
Buckets
Azure
Oracle
Excellent project and time manager
outstanding excellent presentation skills
a great team player
Data Engineer | TC Energy
Present | Odfjell Terminal
TX | 02/2019
09/2019 | Enterprise 
Products
CF
process data pipeline
asset data migration
Amazon Kinesis
Firehose
storage
Snowflake
bucket
Amazon Managed Workflows
Programming language
Python
SparkSql
data pipeline
Zeppelin Notebook
Programming language
Python
SQL 
• 
Created DataStream
AWS kinesis DataStream
Programming language
Python
SparkSql 
• 
Migration solution
• 
Solved data quality issues
AWS EMR
Databricks
• 
AWS Project monitoring
AWS Lambda
IG
TX |
04/2017
process data pipeline
structured data
quantitative and 
qualitative asset ri

In [122]:
df

,skills,experience,education,miscellaneous,achievements
0,"[software, languages, and tools with the abili...","[ , , , , , Data Engineer | TC Energy | Ho...","[ , , , University of North Texas | Master o...",NaN,"[ , , , Licensed Texas Professional Engineer..."


In [123]:
%%time
final_designation = '\n'.join(df['experience'][0])
final_designation

CPU times: user 131 µs, sys: 8 µs, total: 139 µs
Wall time: 144 µs


' \n \n \n \n \nData Engineer | TC Energy | Houston, TX | 09/2019– Present | Odfjell Terminal | Seabrook, TX | 02/2019– 09/2019 | Enterprise \nProducts (Through CF) | Houston, TX | 10/2018 – 02/2019 \n• \nDesign and implementation of process data pipeline for asset data migration into AWS cloud environment \n• \nBuilt Snowflake Data Pipeline using Amazon Kinesis Firehose starting from the AWS EC2 logs to storage in Snowflake \nand AWS S3 bucket post-transformation and orchestrating through Amazon Managed Workflows for Apache Airflow \n(MWAA) DAGs.. Programming language used are Pyspark, Python, SparkSql \n• \nArchitecting data pipeline with Cloudera infrastructure – Ambari, Hortonworks Sandbox using Zeppelin Notebook. \nProgramming language used are Python, SQL \n• \nCreated DataStream using AWS kinesis DataStream and kinesis firehose. Programming language used are Pyspark, \nPython, SparkSql \n• \nMigration solution from SAP and Maximo to Data Lake. \n• \nSolved data quality issues us

# Find Job titles in resume

In [124]:
%%time
store_designations = []

nlp_model_d = spacy.load('/content/drive/MyDrive/hackathon/designation_model')
doc = nlp_model_d(final_designation)
for ent in doc.ents:
    store_designations.append(ent.text)
    print(f'{ent.label_.upper():{30}}- {ent.text}')

DESIGNATION                   - Senior Data Analyst
DESIGNATION                   - Project Engineer
CPU times: user 773 ms, sys: 30 ms, total: 803 ms
Wall time: 973 ms


In [125]:
%%time
store_designations

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


['Senior Data Analyst', 'Project Engineer']

# O-Net mapping for Standardised Job Title

In [175]:
%%time
# Get the file path
file_path = '/content/drive/MyDrive/hackathon/data.csv'

# Read the CSV file into a DataFrame
df_ = pd.read_csv(file_path)

# Get the desired column as a list
labels = df_['Title'].tolist()
codes = df_['O*NET Code'].tolist()
res = dict(zip(labels, codes))

CPU times: user 15.3 ms, sys: 985 µs, total: 16.3 ms
Wall time: 599 ms


In [127]:
%%time
model2 = SentenceTransformer('all-MiniLM-L6-v2')

CPU times: user 276 ms, sys: 49.2 ms, total: 326 ms
Wall time: 2.21 s


In [128]:
%%time
standard_job_titles = labels
standard_embeddings = model2.encode(standard_job_titles)

CPU times: user 5.26 s, sys: 22 ms, total: 5.28 s
Wall time: 5.29 s


In [129]:
%%time
extracted_job_titles = store_designations
extracted_embeddings = model2.encode(extracted_job_titles)

CPU times: user 27.5 ms, sys: 969 µs, total: 28.5 ms
Wall time: 28.3 ms


In [130]:
%%time
# Function to find the most similar standard job title for each extracted job title
def find_most_similar_jobs(extracted_embeddings, standard_embeddings, standard_job_titles):
    for extracted_embedding in extracted_embeddings:
        similarities = util.cos_sim(extracted_embedding, standard_embeddings)
        max_index = np.argmax(similarities)

        yield standard_job_titles[max_index]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [131]:
%%time
# Mapping each extracted job title to the most similar standard job title
mapped_job_titles = list(find_most_similar_jobs(extracted_embeddings, standard_embeddings, standard_job_titles))
for extracted, mapped in zip(store_designations, mapped_job_titles):
    print(f'Extracted: {extracted} -> Mapped: {mapped}')

Extracted: Senior Data Analyst -> Mapped: Data Scientists
Extracted: Project Engineer -> Mapped: Project Management Specialists
CPU times: user 8.51 ms, sys: 0 ns, total: 8.51 ms
Wall time: 23 ms


# Extracting Education

In [88]:
%%time
final_education = '\n'.join(df['education'][0])
final_education

CPU times: user 155 µs, sys: 0 ns, total: 155 µs
Wall time: 162 µs


'Nirma University\nAhmedabad, In\nB.Tech Computer Science Engineering\n2021-Present\nCumulative GPA: 8.30/10; Certificate of Scholar, 3rd Semester\nPuna International School\nAhmedabad, In\nHSC, CBSE: 85.4%\n2021'

# Extracting Timeline

{'2013-09-02': 'Project Engineer', '2017-04-02': 'Senior Data Analyst'}
CPU times: user 500 µs, sys: 0 ns, total: 500 µs
Wall time: 508 µs


{'2013-09-02': 'Project Management Specialists', '2017-04-02': 'Data Scientists'}


# Mapping description and extracting skills for each job title

In [160]:
%%time
s=df['experience'][0]
s=' '.join(s)
l=[0]
for i in store_designations:
  l+=[s.index(i)]
l+=[len(s)]
l
se=[]
for i in range(1,len(l)-1):
  se+=[s[l[i]:l[i+1]]]
sk=[]
for d in se:
  sk+=[extract_skills(d)]
sk

1/1 [==============================] - 0s 32ms/step


[['Senior Data Analyst | Tesoro Logistics',
  'IG',
  'TX |',
  '04/2017',
  '10/2018    •  Design',
  'implementation',
  'process data pipeline',
  'structured data',
  'quantitative and  qualitative asset risk modelling',
  'predictive analysis',
  'forecast'],
 ['wireline instrumentations',
  'well depth',
  'true vertical height',
  'drilling bit',
  'a ‘.csv’ data file',
  'Load data',
  'Weatherford microflux database',
  'analysis',
  '•  Project management',
  'secure drilling operations',
  'Iraq',
  'Angola',
  'clients',
  'Repsol',
  'Exxon Mobile',
  'Total']]

In [161]:
%%time
skill_preds=[]
for skill in sk:
  temp='\n'.join(skill)
  doc=nlp_model(temp)
  temp_skills=''
  for ent in doc.ents:
   temp_skills+=ent.text
  skill_preds.append(temp_skills)

# Resume Summary

In [154]:
df

,skills,experience,education,miscellaneous,achievements
0,"[software, languages, and tools with the abili...","[ , , , , , Data Engineer | TC Energy | Ho...","[ , , , University of North Texas | Master o...",NaN,"[ , , , Licensed Texas Professional Engineer..."


In [136]:
%%time
raw_exp='\n'.join(df.iloc[0]['experience'])
raw_exp

' \n \n \n \n \nData Engineer | TC Energy | Houston, TX | 09/2019– Present | Odfjell Terminal | Seabrook, TX | 02/2019– 09/2019 | Enterprise \nProducts (Through CF) | Houston, TX | 10/2018 – 02/2019 \n• \nDesign and implementation of process data pipeline for asset data migration into AWS cloud environment \n• \nBuilt Snowflake Data Pipeline using Amazon Kinesis Firehose starting from the AWS EC2 logs to storage in Snowflake \nand AWS S3 bucket post-transformation and orchestrating through Amazon Managed Workflows for Apache Airflow \n(MWAA) DAGs.. Programming language used are Pyspark, Python, SparkSql \n• \nArchitecting data pipeline with Cloudera infrastructure – Ambari, Hortonworks Sandbox using Zeppelin Notebook. \nProgramming language used are Python, SQL \n• \nCreated DataStream using AWS kinesis DataStream and kinesis firehose. Programming language used are Pyspark, \nPython, SparkSql \n• \nMigration solution from SAP and Maximo to Data Lake. \n• \nSolved data quality issues us

# Replacing Job Titles with Standard O*Net Job Titles

In [140]:
%%time
def replace_strings(work_experience, original_strings, replacement_strings):
    """
    Replace occurrences of strings in 'original_strings' with corresponding strings in 'replacement_strings' in the 'work_experience' text.

    Parameters:
    - work_experience (str): The text containing the work experience.
    - original_strings (list of str): Strings to be replaced.
    - replacement_strings (list of str): Strings to replace with.

    Returns:
    - str: Updated work experience text.
    """
    for original, replacement in zip(original_strings, replacement_strings):
        work_experience = work_experience.replace(original, replacement)
    return work_experience

In [141]:
%%time
final_exp = replace_strings(raw_exp, store_designations, mapped_job_titles)

In [137]:
%%time
for designation, standard_designation in store_designations, mapped_job_titles:
  if designation in raw_exp:
    raw_exp.replace(designation, standard_designation)

In [151]:
%%time
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [171]:
%%time
import re

def time_line():
  date_pattern = r'\b(?:\d{2}/\d{4}|\d{2}/\d{2}|\d{2}-\d{2}-\d{2}|\d{2}-\d{2}-\d{4})\b'


  #for designation
  date_with_designation = []
  for designations in store_designations:
    index = final_designation.index(designations)
    temp_str = final_designation[index : ]

    dates = re.findall(date_pattern, temp_str)
    if(len(dates) > 0):
      date_with_designation.append(dates[0])





  return date_with_designation

des_date = time_line()

from dateutil.parser import parse

# print(mapped_titles)
# print(des_date)


time = dict(zip(des_date, store_designations))
dates = [parse(date_str) for date_str in des_date]
sorted_dates_with_titles = sorted(zip(dates, store_designations))
sorted_dates = [date.strftime('%Y-%m-%d') for date, _ in sorted_dates_with_titles]
sorted_titles = [title for _, title in sorted_dates_with_titles]
sorted_time = dict(zip(sorted_dates, sorted_titles))

# print(sorted_time)

{'2013-09-02': 'Project Engineer', '2017-04-02': 'Senior Data Analyst'}
CPU times: user 1.68 ms, sys: 0 ns, total: 1.68 ms
Wall time: 1.66 ms


In [177]:
%%time
print('----------------------------')
print('RESUME DETAILS')
print('----------------------------')
print('----------------------------')
print('EDUCATION')
print('----------------------------')
print('\n'.join(df.iloc[0]['education']))
print('----------------------------')
print('----------------------------')
print('EXPERIENCE')
print('----------------------------')
print('DESIGNATIONS AS PER O*NET DATABASE:')
print('----------------------------')
# print('\n'.join(mapped_job_titles))
for jtitle, desc, skill_pred in zip(mapped_job_titles,se,skill_preds):
  print('***TITLE***')
  print(f'{jtitle} --> {res[jtitle]}')
  print('***DESCRIPTION***')
  print(desc)
  print('***PREDICTED SKILLS***')
  print(skill_pred)
  print('\n')

print('----------------------------')
# print('\n'.join(df.iloc[0]['experience']))
# print(final_exp.strip())
print('----------------------------')
print('ABSTRACTIVE SKILLS')
print('----------------------------')
print(abstractive_skills)
print('----------------------------')
print('----------------------------')
print('SKILLS')
print('----------------------------')
print('\n'.join(df.iloc[0]['skills']))
print('----------------------------')
print('----------------------------')
print('ACHIEVEMENTS')
print('----------------------------')
print('\n'.join(df.iloc[0]['achievements']))
print('----------------------------')
print('----------------------------')
print('CAREER TRAJECTORY')
print('----------------------------')
for key in sorted_time:
  print(f'{key}--->{sorted_time[key]}')
print('----------------------------')
print('----------------------------')



----------------------------
RESUME DETAILS
----------------------------
----------------------------
EDUCATION
----------------------------
 
 
 
University of North Texas | Master of Science | Mechanical and Energy Engineering| Denton, TX 
Rice University | Certificate | Data Analytics | Houston, TX  
 
----------------------------
----------------------------
EXPERIENCE
----------------------------
DESIGNATIONS AS PER O*NET DATABASE:
----------------------------
***TITLE***
Data Scientists --> 15-2051.00
***DESCRIPTION***
Senior Data Analyst | Tesoro Logistics (Through IG) | San Antonio, TX | 04/2017 – 10/2018    •  Design and implementation of process data pipeline and loading of structured data for performing quantitative and  qualitative asset risk modelling, predictive analysis for program budgeting and forecast.  •  Created process safety risk reduction metrics  
***PREDICTED SKILLS***



***TITLE***
Project Management Specialists --> 13-1082.00
***DESCRIPTION***
Project Engine